# Install packages

In [ ]:
# Install dependencies
# !pip3 uninstall numpy
!pip3 install --upgrade numpy==2.0.0
!pip3 install pandas
!pip3 install scikit-learn mlflow seaborn shap
!pip3 install bayesian-optimization
!pip3 install xgboost==2.1.2
!pip3 install optuna
!pip3 install optuna-integration[mlflow]


# Import package

In [10]:
# Import library

import pandas as pd
import numpy as np

import mlflow
from mlflow import MlflowClient
from mlflow.models import infer_signature, make_metric
from optuna.integration.mlflow import MLflowCallback
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, plot_importance
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import optuna
import os, json

# Prepare dataset
Because test dataset not have label, we must split train dataset to 2 parts. One for train and one for validate. We just do this on the first time

In [11]:
# Read dataset
# df = pd.read_csv('data/data.csv')
# train_test_data, validate_data = train_test_split(df, test_size=0.2, random_state=42)
# train_test_data.to_csv('data/train_.csv', index=False, header=True)
# validate_data.to_csv('data/validate.csv', index=False, header=True)

Now we check some information of dataset

In [12]:
# Check dataset
df = pd.read_csv('data/train_.csv')
df.head()
# print("Dataset column")
df.columns
# print("Summary of dataset info")
df.info()
# print("view dimensions of dataset")
df.shape

# for col in df.columns:
#   if df[col].dtype != 'object':  # Exclude non-numeric columns
#     min_val = df[col].min()
#     max_val = df[col].max()
#     print(f"Column: {col}")
#     print(f"Minimum: {min_val}")
#     print(f"Maximum: {max_val}")
#     print()

df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560110 entries, 0 to 1560109
Data columns (total 48 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   ID               1560110 non-null  int64  
 1   flow_duration    1560110 non-null  float64
 2   Header_Length    1404309 non-null  float64
 3   Protocol type    1404300 non-null  float64
 4   Duration         1404067 non-null  float64
 5   Rate             1403930 non-null  float64
 6   Srate            1404035 non-null  float64
 7   Drate            1404061 non-null  float64
 8   fin_flag_number  1560110 non-null  float64
 9   syn_flag_number  1560110 non-null  float64
 10  rst_flag_number  1404080 non-null  float64
 11  psh_flag_number  1404104 non-null  float64
 12  ack_flag_number  1560110 non-null  float64
 13  ece_flag_number  1404221 non-null  float64
 14  cwr_flag_number  1403991 non-null  float64
 15  ack_count        1404032 non-null  float64
 16  syn_count        1

ID                      0
flow_duration           0
Header_Length      155801
Protocol type      155810
Duration           156043
Rate               156180
Srate              156075
Drate              156049
fin_flag_number         0
syn_flag_number         0
rst_flag_number    156030
psh_flag_number    156006
ack_flag_number         0
ece_flag_number    155889
cwr_flag_number    156119
ack_count          156078
syn_count          156278
fin_count               0
urg_count               0
rst_count               0
HTTP               155993
HTTPS              156399
DNS                     0
Telnet             156044
SMTP               155832
SSH                156261
IRC                     0
TCP                156010
UDP                     0
DHCP                    0
ARP                156189
ICMP               155988
IPv                     0
LLC                     0
Tot sum            155800
Min                156138
Max                155961
AVG                     0
Std         

We see some cell have null value, we can not drop which rows have null cell because it to much. So we just fill all null value = -1

In [13]:
# Fill all null data as -1

val_dataset_ = pd.read_csv('data/validate.csv')
val_dataset = val_dataset_.fillna(-1, inplace=False)

print(df.isnull().sum())

data_n_null = df.fillna(-1, inplace=False)
data_n_null.isnull().sum()
data_n_null.head()
data_n_null.duplicated().sum()
data_n_null['Label'].unique().tolist()


X_val = val_dataset.drop(columns=['Label'])
y_val = val_dataset['Label']


ID                      0
flow_duration           0
Header_Length      155801
Protocol type      155810
Duration           156043
Rate               156180
Srate              156075
Drate              156049
fin_flag_number         0
syn_flag_number         0
rst_flag_number    156030
psh_flag_number    156006
ack_flag_number         0
ece_flag_number    155889
cwr_flag_number    156119
ack_count          156078
syn_count          156278
fin_count               0
urg_count               0
rst_count               0
HTTP               155993
HTTPS              156399
DNS                     0
Telnet             156044
SMTP               155832
SSH                156261
IRC                     0
TCP                156010
UDP                     0
DHCP                    0
ARP                156189
ICMP               155988
IPv                     0
LLC                     0
Tot sum            155800
Min                156138
Max                155961
AVG                     0
Std         

For tracking during training, we using MLflow. The software defined by container in mlflow folder

In [14]:
# Set mlflow as tracking server
ML_TRACKING_URL = "http://localhost:5000"
mlflow.set_tracking_uri(ML_TRACKING_URL)

# Train model
We train with some model with these steps
- We training with small part of dataset (0.2 or 0.3): dataset_frac
- We log artifacts, we see some column less contribute in  Feature Importance Score, so we delete it
- We train with full dataset, verify droped column is correct and need modify or not
- We use RandomizedSearchCV to search parameter
- We save best parameter to mlflow. With mlflow.sklearn.autolog, model and its metrics was save to model registry. We just download it and use

## Decision Tree


In [ ]:
tags = {
    "dataset_frac": 1.0,
    "test_size" : 0.2,
    "droped_column" : ['ID','IPv','DNS','IRC','DHCP','ARP','SMTP','cwr_flag_number','ece_flag_number','Telnet','Drate','psh_flag_number','rst_flag_number','LLC', 'TCP','SSH','HTTPS','ack_flag_number','Std','Tot size', 'ack_count'],
    "author": "Son Nguyen"
}

def dct_objective(trial):
    with mlflow.start_run(nested=True) as run:
        # params = {
        #     "max_features": trial.suggest_int("max_features", 30, 100, step=2),
        #     "criterion": trial.suggest_categorical("criterion", ["gini"]),
        #     "max_depth": trial.suggest_int("max_depth", 1000, 1500, step=50),
        #     "min_samples_split": trial.suggest_int("min_samples_split", 18, 30, step=2),
        #     "min_samples_leaf": trial.suggest_int("min_samples_leaf", 10, 16, step = 1),
        #     "random_state" : 42
        # }
        params = {
            "max_features": trial.suggest_int("max_features", 30, 60, step=1),
            "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
            "splitter": trial.suggest_categorical("splitter", ["best", "random"]),
            "splitter": trial.suggest_categorical("splitter", ["best"]),
            "max_depth": trial.suggest_int("max_depth", 1000, 2000, step=50),
            "min_samples_split": trial.suggest_int("min_samples_split", 10, 30, step=1),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 10, 16, step=1),
            "min_impurity_decrease": trial.suggest_float("min_impurity_decrease", 0.0, 0.005, step=0.001),
            "ccp_alpha": trial.suggest_float("ccp_alpha", 0.0, 0.005, step=0.001),
            "random_state": 42
        }
        model = DecisionTreeClassifier(**params)

        mlflow.sklearn.autolog()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)


        feature_scores = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
        plt.figure(figsize=(20, 20))
        sns.barplot(x=feature_scores, y=feature_scores.index)
        plt.xlabel('Feature Importance Score')
        plt.ylabel('Features')
        plt.title("Visualizing Important Features")
        feature_importance_plot = "feature_importance.png"
        plt.savefig(feature_importance_plot, bbox_inches='tight')
        mlflow.log_artifact(feature_importance_plot)
        os.remove(feature_importance_plot)

        
        metrics_file = "model_summary.json"
        metrics = {
            "parameter" : {**params},
            "metrics" : {
                "f1" : f1,
                "precision" : precision,
                "accuracy" : accuracy,
                "recall" : recall
            },
            "droped_column" : [tags["droped_column"]]
            
        }
        
        with open(metrics_file, "w") as f:
            json.dump(metrics, f, indent=4)

        mlflow.log_artifact(metrics_file)
        os.remove(metrics_file)

        trial.set_user_attr("run_id", run.info.run_id)
    return f1


def dct_callback(study, frozen_trial):
    winner = study.user_attrs.get("winner", None)
    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")


data = data_n_null.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = 42)


mlflow.set_experiment("decision_tree")
optuna.logging.set_verbosity(optuna.logging.ERROR)



with mlflow.start_run(nested=True) as run:

    # mlflow.xgboost.autolog() can not put auto log here
    study = optuna.create_study(direction='maximize')
    study.optimize(dct_objective, n_trials=100, timeout=14400, callbacks=[dct_callback], show_progress_bar=True)
    # study.optimize(dct_objective, n_trials=100, timeout=100, callbacks=[dct_callback], show_progress_bar=True)
    
    best_trial = study.best_trial
    best_run_id = best_trial.user_attrs['run_id']
    # best_param = study.best_params
    best_value = study.best_value

    model_name = "Decision Tree"
    client = mlflow.tracking.MlflowClient()
    latest_ = client.get_latest_versions(model_name, stages=None)[0]

    model_uri = f"runs:/{best_run_id}/model"
    best_model = mlflow.register_model(model_uri, model_name)
    
    
    best_param = client.get_run(best_run_id).data.params
    if latest_:
        previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
        if previous_f1_score >= best_value:
            print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
        else:
    
            client.update_registered_model(
                name=model_name,
                description="Best moldel",
            )

            for key, value in best_param.items():
                client.set_model_version_tag(
                    name=model_name,
                    version=best_model.version,
                    key=key,
                    value=value
                )

            client.set_model_version_tag(
                name=model_name,
                version=best_model.version,
                key="values",
                value=best_value
            )

            for key, value in tags.items():
                mlflow.set_tag(key,value)
            mlflow.set_tag("job", "Decision Tree using optuna to search parameter")


  0%|          | 0/100 [00:00<?, ?it/s]

d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


🏃 View run stately-foal-401 at: http://localhost:5000/#/experiments/1/runs/c943d75c06fc4cccbad8628bbcb372ff
🧪 View experiment at: http://localhost:5000/#/experiments/1
Initial trial 0 achieved value: 0.42564423413116004


d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


🏃 View run chill-shark-917 at: http://localhost:5000/#/experiments/1/runs/deada7c52d4a43ebb1a57b188f3c3669
🧪 View experiment at: http://localhost:5000/#/experiments/1
Trial 1 achieved value: 0.6859913046009292 with  37.9519% improvement


...

🏃 View run intelligent-dove-709 at: http://localhost:5000/#/experiments/1/runs/d4034104073b4c859d8a58be6b68c554
🧪 View experiment at: http://localhost:5000/#/experiments/1


C:\Users\SonNH\AppData\Local\Temp\ipykernel_20084\1194028022.py:119: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_ = client.get_latest_versions(model_name, stages=None)[0]
Registered model 'Decision Tree' already exists. Creating a new version of this model...
2024/12/08 03:25:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Decision Tree, version 3


🏃 View run beautiful-hawk-458 at: http://localhost:5000/#/experiments/1/runs/cd019c9a679844fabf7e82f15f1aaa91
🧪 View experiment at: http://localhost:5000/#/experiments/1


Created version '3' of model 'Decision Tree'.


🏃 View run beautiful-fly-412 at: http://localhost:5000/#/experiments/1/runs/fbdf166db8b247c5ba6d4b420edd4241
🧪 View experiment at: http://localhost:5000/#/experiments/1


## XGBoost

In [16]:
tags = {
    "dataset_frac": 1.0,
    "random_state": 42,
    "test_size" : 0.2,
    "droped_column" : ['ID','IPv','Drate','Telnet','SMTP','ARP','cwr_flag_number','ece_flag_number','fin_flag_number','SSH','psh_flag_number','rst_flag_number'],
    "author": "Son Nguyen"
}

def xgboost_objective(trial):
    with mlflow.start_run(nested=True) as run:
        params = {
            "tree_method" : "hist",
            "device" : "cuda",
            "objective": "reg:squarederror",
            "n_estimators": 1000,
            "verbosity": 0,
            "eval_metric" : ["rmse", "mae", "mape", "logloss","error","auc"],
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "max_depth": trial.suggest_int("max_depth", 10, 50),
            "subsample": trial.suggest_float("subsample", 0.5, 1),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.8),
            "min_child_weight": trial.suggest_int("min_child_weight", 1,15),
        }
        model = XGBClassifier(**params)
        mlflow.xgboost.autolog()
        model.fit(X_train, y_train, verbose=False)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)

        metrics_file = "model_summary.json"
        metrics = {
            "parameter" : {**params},
            "metrics" : {
                "f1" : f1,
                "precision" : precision,
                "accuracy" : accuracy,
                "recall" : recall
            },
            "droped_column" : [tags["droped_column"]]
            
        }
        
        with open(metrics_file, "w") as f:
            json.dump(metrics, f, indent=4)

        mlflow.log_artifact(metrics_file)
        os.remove(metrics_file)

        trial.set_user_attr("run_id", run.info.run_id)
    return f1

def champion_callback(study, frozen_trial):
    winner = study.user_attrs.get("winner", None)
    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

data = data_n_null.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = tags['random_state'])

mlflow.set_experiment("xgboost")
optuna.logging.set_verbosity(optuna.logging.ERROR)
with mlflow.start_run(nested=True) as run:

    # mlflow.xgboost.autolog() can not put auto log here
    study = optuna.create_study(direction='maximize')
    study.optimize(xgboost_objective, n_trials=100, timeout=14400, callbacks=[champion_callback], show_progress_bar=True)
    
    best_trial = study.best_trial
    best_run_id = best_trial.user_attrs['run_id']
    # best_param = study.best_params
    best_value = study.best_value

    model_name = "XGBoost-Classifier"
    client = mlflow.tracking.MlflowClient()
    latest_ = client.get_latest_versions(model_name, stages=None)[0]

    model_uri = f"runs:/{best_run_id}/model"
    best_model = mlflow.register_model(model_uri, model_name)
    
    best_param = client.get_run(best_run_id).data.params
    if latest_:
        previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
        if previous_f1_score >= best_value:
            print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
        else:
            client.update_registered_model(
                name=model_name,
                description="Best moldel",
            )

            for key, value in best_param.items():
                client.set_model_version_tag(
                    name=model_name,
                    version=best_model.version,
                    key=key,
                    value=value
                )

            client.set_model_version_tag(
                name=model_name,
                version=best_model.version,
                key="values",
                value=best_value
            )

            for key, value in tags.items():
                mlflow.set_tag(key,value)
            mlflow.set_tag("job", "xgboost using optuna to search parameter")
    


  0%|          | 0/100 [00:00<?, ?it/s]

2024/12/08 03:34:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [03:34:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run learned-cub-276 at: http://localhost:5000/#/experiments/2/runs/03501db7b5b34c8a8ba58d80fda4b4fb
🧪 View experiment at: http://localhost:5000/#/experiments/2
Initial trial 0 achieved value: 0.9403774997714633


2024/12/08 03:55:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [03:55:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run tasteful-shrike-439 at: http://localhost:5000/#/experiments/2/runs/26b5086f196b49198adc4a19fc74ab76
🧪 View experiment at: http://localhost:5000/#/experiments/2
Trial 1 achieved value: 0.94038969899047 with  0.0013% improvement


...

🏃 View run placid-conch-14 at: http://localhost:5000/#/experiments/2/runs/2ea8d0312c6f4b9697bca5aa653c7182
🧪 View experiment at: http://localhost:5000/#/experiments/2


2024/12/08 07:31:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [07:31:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run enchanting-snake-544 at: http://localhost:5000/#/experiments/2/runs/9049bfbaf61544f0b308035d2d2426bd
🧪 View experiment at: http://localhost:5000/#/experiments/2


C:\Users\SonNH\AppData\Local\Temp\ipykernel_20084\2784243908.py:95: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_ = client.get_latest_versions(model_name, stages=None)[0]
Registered model 'XGBoost-Classifier' already exists. Creating a new version of this model...
2024/12/08 07:31:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoost-Classifier, version 3
Created version '3' of model 'XGBoost-Classifier'.


🏃 View run big-dog-621 at: http://localhost:5000/#/experiments/2/runs/cbcc7a2f5e7e4ae6ada0a5e0aaaa5ee7
🧪 View experiment at: http://localhost:5000/#/experiments/2
